<a href="https://colab.research.google.com/github/RafaelBNN/if697/blob/main/Projeto_2_IF697.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A ideia vai ser prever o `Place` de um atleta de acordo com `Age`, `BodyweightKg`, `Sex`, `Division`, `IPFPoints` e `Place` de atletas com atributos semelhantes. O professor sugeriu coletar uma amostra dos últimos 5 anos do dataset original e usar 3,5 anos como *training* set, 0,5 anos como *validation* set e o último ano como *test* set.

*Na real eu não entendi se é pra prever a colocação de um atleta de acordo com colocações anteriores deste mesmo atleta ou de acordo com colocações anteriores de atletas semelhantes*



*Atualização 27/07*: Acho que esse problema se aplica mais como um problema de regressão, já que não precisamos prever com 100% de certeza a colocação de um competidor. A parte decimal das predições vai nos dar uma ideia da "certeza" do algoritmo de que o competidor vai ficar naquela colocação.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/RafaelBNN/if697/main/powerlifting_final2.csv?token=AKYEI53FXBYDBCB4LYZUI6DBADBWQ")

In [3]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Name,Sex,Event,Equipment,Age,AgeClass,Division,BodyweightKg,WeightClassKg,Squat1Kg,Squat2Kg,Squat3Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,Best3BenchKg,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Best3DeadliftKg,TotalKg,Place,Wilks,McCulloch,Glossbrenner,IPFPoints,Tested,Country,Federation,Date,MeetCountry,MeetState,MeetName,Best3SquatKg_norm,Best3BenchKg_norm,Best3DeadliftKg_norm,TotalKg_norm,BodyweightKg_norm,IPFPoints_norm,TotalKg_disc,lof_outlier
0,0,674206,Dane Roach,1,SBD,Raw,25.500000,24-34,MR-O,81.20,83,157.5,167.5,175.0,175.00,117.5,122.5,127.5,127.50,205.0,220.0,227.5,227.5,530.0,3,358.49,358.49,346.20,525.08,Yes,NaN,USAPL,2017-12-02,USA,MI,Michigan State Meet,-0.340206,-0.409639,0.051948,-0.148760,-0.344148,-0.167948,"(487.5, 551.11]",1
1,1,507347,Cody Robert,1,SBD,Single-ply,23.744319,NaN,Boys,141.52,124.7+,NaN,NaN,NaN,204.12,NaN,NaN,NaN,136.08,NaN,NaN,NaN,183.7,523.9,7,292.29,292.29,276.78,345.78,Yes,NaN,THSPA,2019-02-16,USA,TX,Rice Consolidated Booster Club Meet,-0.220124,-0.368289,-0.175584,-0.158843,0.321525,-0.488504,"(487.5, 551.11]",1
2,2,1054061,Satu Luoto,0,SBD,Raw,46.500000,45-49,Masters 1,62.76,63,122.5,127.5,-132.5,127.50,82.5,85.0,-87.5,85.00,125.0,130.0,-132.5,130.0,342.5,4,368.90,393.98,325.76,593.66,Yes,Finland,IPF,2016-06-19,USA,TX,World Classic Powerlifting Championships,-0.536082,-0.614458,-0.454545,-0.458678,-0.547647,-0.045339,"(308.44, 360.0]",1
3,3,587550,Andreas Andersson,1,SBD,Raw,34.410717,NaN,Juniors 19-23,101.50,105,160.0,170.0,180.0,180.00,120.0,125.0,NaN,125.00,220.0,230.0,-235.0,230.0,535.0,1,323.70,323.70,310.96,456.95,Yes,NaN,NSF,2016-03-02,Norway,NaN,Klubbmesterskap,-0.319588,-0.421687,0.064935,-0.140496,-0.120124,-0.289752,"(487.5, 551.11]",1
4,4,227497,Ben Leyland,1,SBD,Raw,28.671584,NaN,M-C-U23,107.20,120,195.0,207.5,-217.5,207.50,130.0,-132.5,-132.5,130.00,250.0,277.5,290.0,290.0,627.5,2,372.32,372.32,357.01,532.45,Yes,NaN,EPA,2015-07-18,England,NaN,East Midlands Masters Juniors and Novice Champ...,-0.206186,-0.397590,0.376623,0.012397,-0.057220,-0.154772,"(551.11, 632.5]",1


In [4]:
print('Numero de registros de 2015:', len(df[(df['Date']>'2015-01-01')&(df['Date']<'2016-01-01')]))
print('Numero de registros de 2016:', len(df[(df['Date']>'2016-01-01')&(df['Date']<'2017-01-01')]))
print('Numero de registros de 2017:', len(df[(df['Date']>'2017-01-01')&(df['Date']<'2018-01-01')]))
print('Numero de registros de 2018:', len(df[(df['Date']>'2018-01-01')&(df['Date']<'2019-01-01')]))
print('Numero de registros de 2019:', len(df[(df['Date']>'2019-01-01')&(df['Date']<'2020-01-01')]))

Numero de registros de 2015: 3093
Numero de registros de 2016: 3551
Numero de registros de 2017: 4062
Numero de registros de 2018: 4331
Numero de registros de 2019: 1800


Dessa forma, vamos utilizar os anos de 2015 a 2018 para *training* e *validation set* e os registros de 2019 para *test set*.